# <span style="color:blue">First steps with `exo_k`</span>

*Author: Jeremy Leconte (CNRS/LAB/Univ. Bordeaux)*

The goal of `exo_k` is to provide a library to:

* Interpolate efficiently and easily in correlated-k and cross section tables.
* Convert easily correlated-k and cross section tables from one format to another (pickle, hdf5, LMDZ GCM format).
* Adapt precomputed correlated-k tables to your needs by changing:
  * the resolution and quadrature (g) grid,
  * the pressure/temperature grid.
* Create tables for a mix of gases using tables for individual gases.
* Create your own tables from high-resolution spectra.
* Use your data in an integrated radiative transfer framework to simulate planetary atmospheres.
   
This tutorial will show you how to do all that.



## Initialization

First, let's make some common initializations.   
We also import our library: `exo_k`. For this to work, you need to have installed the library by typing
```pip install -e .```
at the root of exo_k directory (the location where you first saw this tutorial).

In [1]:
import exo_k as xk
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import time,sys,os

In [2]:
# Uncomment the line below if you want to enable interactive plots
#%matplotlib notebook
plt.rcParams["figure.figsize"] = (7,4)
from matplotlib import cycler
colors = cycler('color',[plt.cm.inferno(i) for i in np.linspace(0.1,1,5)])
plt.rc('axes', axisbelow=True, grid=True, labelcolor='dimgray', labelweight='bold', prop_cycle=colors)
plt.rc('grid', linestyle='solid')
plt.rc('xtick', direction='in', color='dimgray')
plt.rc('ytick', direction='in', color='dimgray')
plt.rc('lines', linewidth=1.5)

## Documentation

The documentation in html form can be found at http://perso.astrophy.u-bordeaux.fr/~jleconte/exo_k-doc/index.html

It is searchable, and has a general index to find any function.

To access it through a python interface, just run the following line for any class, function, etc.:

In [ ]:
help(xk.Ktable().bin_down)

# <span style="color:blue">Dealing with `Ktable()` objects</span>

## Loading and saving a `Ktable()` object

For this tutorial to work, you should launch this notebook from a base_directory that contains a `data/corrk/` directory where your correlated k files are storred. A directory with some sample files can be downloaded at the following url:

`https://mycore.core-cloud.net/index.php/s/w2cHuigAiwcfBVW`

Many thanks to K. Chubb and the Exomol project for providing these data. 

### Loading a `Ktable` from a file 

One of the main objects we will deal with is the `Ktable()` object. This contains a big matrix with k-coefficients for a species or a mix of species along with all the needed supporting information such as the Pressure, Temperature, wavenumber, and g grids onto which the k-coefficients have been computed.  

To instantiate a `Ktable()` object from most currently supported formats, just run the following line. If you are specifying the file through the `filename` keyword, you have to give the full path to the file. The format should be recognized from the extension. 

In [ ]:
h2o_ktab=xk.Ktable(filename='data/corrk/H2O_R300_0.3-50mu.ktable.TauREx.pickle')
print(h2o_ktab)

### Managing Units

#### Tracking units

Except for self-defining formats (such as hdf5), most formats handled by `exo_k` do not carry the information on units with them (Exo_transmit, HITRAN .cia, etc.).

However, previous experience has shown us that tracking units is essential to avoid errors and working with SI units is recommended when possible. 

For this reason, the units for pressure, kdata, and wavenumbers are kept as attributes of any Ktable object. They can be seen by printing the object (as above).
If the input format is self-defining (e.g. hdf5) the units are read in the file. For all the other formats, the units are assumed to always be the same and have been inferred from reading the codes using these formats and their documentation (see http://perso.astrophy.u-bordeaux.fr/~jleconte/exo_k-doc/units.html for some examples and a more general discussion).

#### Overriding default units in a file

If for some reason you know that the units used in a given input file are different from the default ones,
you can always override the default units by explicitly stating what are the pressure (`file_p_unit` keyword) and k-coefficient (`file_kdata_unit` keyword) units.

Let's say, for example, that you know the pressures in the above file where in fact given in mbar, you could specify it using

In [ ]:
xk.Ktable(filename='data/corrk/H2O_R300_0.3-50mu.ktable.TauREx.pickle', file_p_unit='mbar')

Notice how the code pressure unit as changed, but not the actual values in the pressure grid. This is normal, you did not ask for a conversion, you just specified what units you were expecting from the file. 

#### Converting to new units

To control the unit we want to work with, we just need to specify what units we want with the `p_unit` and `kdata_unit` keywords.

These keywords accept string input with units recognized by the `astropy.units` library such as 'Pa', 'mbar', and 'bar' for pressure and e.g. 'm^2/molecule' and 'cm^2/molecule' for cross sections. Although '/molecule' is not a unit recognized by `astropy.units`, it is automatically appended to the unit string provided by the user when necessary as a reminder that opacities per 'moles' or 'kg' are not supported.

To use SI units, just reload your ktable as shown below. Note that you can also force SI units to be used with the global option to be set once and for all.
```python
xk.Settings().set_mks(True)
```

In [ ]:
h2o_ktab_SI=xk.Ktable(filename='data/corrk/H2O_R300_0.3-50mu.ktable.TauREx.pickle',
                         kdata_unit='m^2', p_unit='Pa')
# or simply
xk.Settings().set_mks(True)
h2o_ktab_SI=xk.Ktable(filename='data/corrk/H2O_R300_0.3-50mu.ktable.TauREx.pickle')
print(h2o_ktab_SI)

Now both the pressure unit and the pressure grid have changed!

### Setting up the search path and searching files using regular expressions

At some point you might get tired of always typing the whole path to your files. To avoid that, we can tell `exo_k` where to look for files. By default, it searches the local directory where the library has been imported ("."), as can be seen by running:

In [ ]:
xk.Settings().reset_search_path()
xk.Settings().search_path()

A new path can be added to the search path as follows

In [3]:
xk.Settings().add_search_path('data/xsec','data/corrk')

In [ ]:
xk.Settings().search_path()

If you do not want to add a path, but to reset the path (in order for exo_k not to see some files in the local directory for example), you can do it as follows:

In [ ]:
xk.Settings().set_search_path('data/corrk')
xk.Settings().search_path()

Notice that these methods can take as many directories as you want following one of the syntaxes below

In [ ]:
xk.Settings().set_search_path('.','data/xsec','data/corrk')
print(xk.Settings().search_path())
xk.Settings().set_search_path(*['.','data/xsec','data/corrk'])
print(xk.Settings().search_path())
xk.Settings().set_search_path(*('.','data/xsec','data/corrk'))
print(xk.Settings().search_path())

Once we have done that, we can only give part of the name of the file, possibly in bits and pieces, as long as it is specific enouh to identify a unique file.

In [ ]:
h2o_ktab_SI=xk.Ktable('H2O_R300_0.3-50mu.ktable.TauREx')
print('file found:',h2o_ktab_SI.filename)

h2o_ktab_SI=xk.Ktable('H2O_R300_0\.3\-50mu\.ktable\.TauREx')
print('file found:',h2o_ktab_SI.filename)

h2o_ktab_SI=xk.Ktable('H2O','R300.*tauREx')
print('file found:',h2o_ktab_SI.filename)

h2o_ktab_SI=xk.Ktable('H2','R300','tauREx')
print('file found:',h2o_ktab_SI.filename)

try:
    h2o_ktab_SI=xk.Ktable('R300','tauREx',mol='H2')
except:
    print("""Notice how the previous statement was looking for 'H2',
    but settled for H2O because the string 'H2O' contains 'H2'.
    If you want to avoid that, really specify a molecule with the mol keyword.""")


By default, the filters you provide are compared to the file names in a case insensitive way to enable more flexibility. This can be changed with

```python
xk.Settings().set_case_sensitive(True)
```

### Saving a Ktable to a pickle or hdf5 file 

To save any `Ktable()` object to a pickle or hdf5 file, just use `write_pickle()` or `write_hdf5()`.
Notice that the full path (relative or absolute) must be provided, except for the file extension that will be added automatically. Here we will write an hdf5 file (more optimal than pickle)

In [ ]:
h2o_ktab_SI=xk.Ktable('H2O_R300_0.3-50mu.ktable.TauREx', kdata_unit='m^2', p_unit='Pa')
h2o_ktab_SI.write_hdf5('data/corrk/H2O_R300_0.3-50mu.ktable.SI')

This new file has saved the unit information. So now, it can be loaded with only:

In [4]:
h2o_ktab_SI=xk.Ktable('H2O_R300_0.3-50mu.ktable.SI')

In [ ]:
h2o_ktab_SI

### What is inside a Ktable object

To see what is inside a `Ktable()` object, you can list the attributes using `__dict__`

.. warning::
    Although you can access and see these attributes, they should NOT be changed manually
    as changing one of them usually implies that others need to be changed too. There is a
    method to change all the attributes that can possibly be changed.

The most useful attributes are the following:  

 * `mol`, Name of the mol described  
 * `Ng`, Number of g points  
 * `filename`,  Name of the file used to load the Ktable  
 * `ggrid`, Abscissa of the g grid  
 * `weights`, quadrature weights corresponding to the g points
 * `kdata`, Numpy array with the coeficients
 * `kdata_unit`, Units of the kdata  
 * `pgrid`, pressure grid  
 * `logpgrid`, Log 10 of the pressure grid  
 * `Np`, Number of pressure points  
 * `p_unit`, Units of the pressure grid  
 * `tgrid`, Temperature grid  
 * `Nt`, Number of Temperature points  
 * `wns`, Central points of the wavenumber bins (in cm^-1)  
 * `wnedges`, Edges of the wavenumber bins (in cm^-1)  
 * `wls`, Wavelengths (10000./wns in microns)  
 * `wledges`, Wavelengths (10000./wnedges in microns)  
 * `Nw`, Number of wavenumber points  
 * `shape`, shape of the kdata array  


In [ ]:
print(h2o_ktab_SI.mol)
print(h2o_ktab_SI.pgrid)

the data themselves can also be accessed directly as follows (but they cannot be modified this way):

In [ ]:
print(h2o_ktab_SI.kdata[0,0,0,:])
print(h2o_ktab_SI[0,0,0,:])

### Handling zeros

At wavelengths where data are not available, the ktable can be filled with zeros that can become a problem for some codes (e.g. because of log interpolation).  

If you want to avoid that, we have created the `remove_zeros=True` option that will replace zeros with a number 10 orders of magnitude smaller than the smallest number in the table. 
```python
h2o_ktab_SI=xk.Ktable(filename, remove_zeros=True, **kwargs)
```
You can also call manually the `remove_zeros()` method on a `Ktable()`object. 

### Copying a Ktable object

If for some reason, as we will do below, you need to copy a Ktable object before modifying it inplace (to change resolution for example), you can use the `copy` method, that is a bit like the deep copy in `numpy`.

In [ ]:
new_h2o_ktab=h2o_ktab_SI.copy()
new_h2o_ktab

Sometimes one needs to copy only the structure and metadata, but not the big `self.kdata` table. Then, you can use:
```
new_h2o_ktab=h2o_ktab_SI.copy(cp_kdata=False)
```

### Clipping data to a given spectral range

If for effciency reasons you want to only deal with a small part of the spectral domain, you can use the `Ktable.clip_spectral_range(wl_range= , wn_range= )` method with either a wavelength range (`wl_range`in microns) or a wavenumber range (`wn_range` in cm^-1).
Data outside this range will be removed from the table. 

## Interpolating and remapping opacities

### Basic interpolation 

To get the value of the cross section of the molecule or mix described by our `Ktable()` object, we can use:

```python
self.interpolate_kdata(logp_array=None,t_array=None,log_interp=None,wngrid_limit=None)
```
This returns the cross section of the molecule along a LogP-T profile specified by `logp_array` and `t_array`. This returns an array of shape `(logp_array.size, self.Nw, self.Ng)`,  i.e (number of PT points, number of wavenumber points, number of g-points). `logp_array`and `t_array`must have the same size.  

It also works if `logp_array` and/or `t_array` are `floats`
    
.. important:: Notice that the input is always in Log10(Pressure) because this is the quantity used for the interpolation

In [ ]:
logp=3.
Temp=1000.
tmp_opa=h2o_ktab_SI.interpolate_kdata(logp,Temp)
print(tmp_opa.shape)
print(tmp_opa)

In [ ]:
logp_array=np.linspace(1.,6.,6)
t_array=[1000. for logp in logp_array]
tmp_opa=h2o_ktab_SI.interpolate_kdata(logp_array,t_array)
print(tmp_opa.shape)

### Interpolation options

The default interpolation scheme interpolates `log(kdata)`. This can be changed directly using the `log_interp=False` keyword. But the default behavior can also be changed using `Settings().set_log_interp`. The keyword always supercedes the default behavior.

In [ ]:
logp=3.
Temp=1000.
tmp_opa_lin1=h2o_ktab_SI.interpolate_kdata(logp,Temp,log_interp=False)

In [ ]:
xk.Settings().set_log_interp(False)
logp=3.
Temp=1000.
tmp_opa_lin2=h2o_ktab_SI.interpolate_kdata(logp,Temp)
np.sum(tmp_opa_lin2-tmp_opa_lin1)
xk.Settings().set_log_interp(True) ## let's go back to default behavior

Linear interpolation is of course faster, but is believed to be less accurate. 

In [ ]:
logp=3.
Temp=1000.
%timeit -n 1000 -r 2 h2o_ktab_SI.interpolate_kdata(logp,Temp,log_interp=True)
%timeit -n 1000 -r 2 h2o_ktab_SI.interpolate_kdata(logp,Temp,log_interp=False)

If we just want a part of the spectrum, we can specify a **wavenumber** range (in cm^-1) to cover as follows.   
You can see that it goes much faster.

In [ ]:
logp=3.
Temp=1000.
wn_range=[6000.,5000.] # does not need to be sorted
%timeit -n 1000 -r 2 h2o_ktab_SI.interpolate_kdata(logp,Temp,wngrid_limit=wn_range,log_interp=True)
%timeit -n 1000 -r 2 h2o_ktab_SI.interpolate_kdata(logp,Temp,wngrid_limit=wn_range,log_interp=False)
tmp_opa=h2o_ktab_SI.interpolate_kdata(logp,Temp,wngrid_limit=wn_range)
print(tmp_opa.shape)
print(tmp_opa)

### (Log P, T) grid Remapping 

If for computational efficiency reason we just want to use a subset of the (Log P, T) grid, we can remap the table. This will modify the Table in place. So we might want to make a copy before.

In [ ]:
Small_Grid_h2o_ktab=h2o_ktab_SI.copy()
logp_array=np.arange(6.)
t_array=np.linspace(200.,1000.,9)

Small_Grid_h2o_ktab.remap_logPT(logp_array=logp_array,t_array=t_array)
Small_Grid_h2o_ktab

## Plotting opacities and k-distributions

To help you vizualize opacities, the  
```python
plot_spectrum(self,ax,p=1.e-5,t=200.,g=0.,xscale=None,xscale=None,x=1.,**kwarg)
```
method is there to make it easy to plot at the requested (p,T,g) point. You can also normalize the cross section 
with the volume mixing ratio you want for the species with the `x=vmr` keyword.

In [ ]:
# %matplotlib notebook
p_plot=1.e5
t_plot=1000.
fig,axs=plt.subplots(2,2,sharex=False,sharey=False,figsize=(9,6.5))
h2o_ktab_SI.plot_spectrum(axs[0,0],p=p_plot,t=t_plot,g=1.,yscale='log',xscale='log',label='g=1')
h2o_ktab_SI.plot_spectrum(axs[0,0],p=p_plot,t=t_plot,g=0.,yscale='log',xscale='log',label='g=0')
axs[0, 0].set_title('Log-Log')
h2o_ktab_SI.plot_spectrum(axs[1,0],p=p_plot,t=t_plot,g=1.,xscale='log',label='g=1')
h2o_ktab_SI.plot_spectrum(axs[1,0],p=p_plot,t=t_plot,g=0.,xscale='log',label='g=0')
axs[1, 0].set_title('Lin-Log')
h2o_ktab_SI.plot_spectrum(axs[0,1],p=p_plot,t=t_plot,g=1.,yscale='log',label='g=1')
h2o_ktab_SI.plot_spectrum(axs[0,1],p=p_plot,t=t_plot,g=0.,yscale='log',label='g=0')
axs[0, 1].set_title('Log-Lin')
h2o_ktab_SI.plot_spectrum(axs[1,1],p=p_plot,t=t_plot,g=1.,label='g=1')
h2o_ktab_SI.plot_spectrum(axs[1,1],p=p_plot,t=t_plot,g=0.,label='g=0')
axs[1, 1].set_title('Lin-Lin')
for axs in axs:
    for ax in axs:
        ax.legend()
#ax.set_ylim(bottom=1.e-37)
fig.tight_layout()

The g gistribution is shown with `plot_distrib`. Notice that when you ask for a log scale in g, we switch to 1-g as it is the most relevant quantity to look at in log.

In [ ]:
p_plot=1.e5
t_plot=1000.
fig,axs=plt.subplots(2,2,sharex=False,sharey=False,figsize=(9,6.5))
h2o_ktab_SI.plot_distrib(axs[0,0],p=p_plot,t=t_plot,wl=1.0,yscale='log',xscale='log',label='$\lambda=1\mu$m')
h2o_ktab_SI.plot_distrib(axs[0,0],p=p_plot,t=t_plot,wl=10.,yscale='log',xscale='log',label='$\lambda=10\mu$m')
h2o_ktab_SI.plot_distrib(axs[1,0],p=p_plot,t=t_plot,wl=1.0,xscale='log',label='$\lambda=1\mu$m')
h2o_ktab_SI.plot_distrib(axs[1,0],p=p_plot,t=t_plot,wl=10.,xscale='log',label='$\lambda=10\mu$m')
h2o_ktab_SI.plot_distrib(axs[0,1],p=p_plot,t=t_plot,wl=1.0,yscale='log',label='$\lambda=1\mu$m')
h2o_ktab_SI.plot_distrib(axs[0,1],p=p_plot,t=t_plot,wl=10.,yscale='log',label='$\lambda=10\mu$m')
h2o_ktab_SI.plot_distrib(axs[1,1],p=p_plot,t=t_plot,wl=1.0,label='$\lambda=1\mu$m')
h2o_ktab_SI.plot_distrib(axs[1,1],p=p_plot,t=t_plot,wl=10.,label='$\lambda=10\mu$m')
for axs in axs:
    for ax in axs:
        ax.legend()
#ax.set_ylim(bottom=1.e-37)
fig.tight_layout()

## Binning down a `Ktable`

We now get to one of the core features of `exo_k`: we want to create a new `Ktable()` object with just the spectral resolution we need in the model we want to run (whether it is a Global Climate Model, a 1D climate model, or a simple forward model for a retrieval).  

You first need to provide a grid, `wnedges`, containing the edges of the new wavenumber bins. So the final Ktable will have a spectral dimension with size `wnedges.size-1`.

You can of course provide any custom grid you want. But if you want a grid at fixed resolution you can use the following
```python
xk.wavenumber_grid_R(min_wavenumber, max_wavenumber, R)
```
where $R=\frac{\nu}{\delta \nu}$.

In [ ]:
R=10
wavenumber_grid=xk.wavenumber_grid_R(200.,10000.,R)
wavenumber_grid

Then we just need to use the `bin_down(wnedges)` method. However, bin_down modifies the `Ktable()` in place. So again, if we want to keep the original one in memory, we just copy it or use `bin_down_cp`.  

Optional keywords can be `weights=` and `ggrid=` where you can provide the weights you want (and in the corresponding g points). In general, for an LMDZ user, just input the values inside g.dat (except for the final zero) in a `w_array`, and use `weights=w_array`. If you want gauss legendre weights, you can have a look at `exo_k.gauss_legendre` in the index.

It should only take a few seconds. (However, the bigger the final resolution wanted, the longer it takes).

In [ ]:
start=time.time()
LowRes_h2o_ktab=h2o_ktab_SI.bin_down_cp(wavenumber_grid)
end=time.time()
print('computation time=',end-start,'s')
LowRes_h2o_ktab

In [ ]:
p_plot=1.e0
t_plot=300.
fig,axs=plt.subplots(1,2,sharex=False,sharey=False)
h2o_ktab_SI.plot_spectrum(axs[0],p=p_plot,t=t_plot,g=1.,yscale='log',xscale='log',label='Original data')
LowRes_h2o_ktab.plot_spectrum(axs[0],p=p_plot,t=t_plot,g=1.,label='Binned data')
h2o_ktab_SI.plot_spectrum(axs[1],p=p_plot,t=t_plot,g=1.,yscale='log',xscale='log',label='Original data')
LowRes_h2o_ktab.plot_spectrum(axs[1],p=p_plot,t=t_plot,g=1.,label='Binned data')
axs[1].set_xlim(left=1.,right=6.)
axs[1].set_xscale('linear')
for ax in axs:
    ax.legend()
fig.tight_layout()

## Rescaling and mixing `Ktable`s

The `Ktable` we have seen up to now contain opacity data (in terms of cross section in surface per molecule) for pure species. To create a mix, you can dilute your tables with (multiply by) the Volume mixing ratio of the relevant species and add them.

.. important::
    Although we are using the '+' sign,
    the tables are actually combined using the Random overlap method.

.. warning::
    `Ktable`s should have the same dimensions and grids to be combined.
    For efficiency, the code only checks whether the dimensions are the same
    (and raises a `TypeError` if they are not).
    The user should make sure the grids are the same as well. 

In [ ]:
h2o=xk.Ktable('H2O_R300_0.3-50mu.ktable.SI', remove_zeros=True)
co2=xk.Ktable('CO2_R300_0.3-50mu.ktable.SI', remove_zeros=True)
for ktab in [co2,h2o]:
    ktab.remap_logPT(logp_array=np.arange(0,5),t_array=np.arange(200.,500.,50.))
start=time.time()
mix=0.01*h2o+co2*0.02
print('Mixing time=',time.time()-start,'s')

In [ ]:
p_plot=1.e0
t_plot=300.
fig,ax=plt.subplots(1,1,sharex=False,sharey=False)
h2o.plot_spectrum(ax,p=p_plot,t=t_plot,g=1.,x=0.01,yscale='log',xscale='log',label='H2O')
co2.plot_spectrum(ax,p=p_plot,t=t_plot,g=1.,x=0.01,label='CO2')
mix.plot_spectrum(ax,p=p_plot,t=t_plot,g=1.,yscale='log',xscale='log',label='mix')
ax.set_ylim(bottom=1.e-36)
ax.legend()
fig.tight_layout()

.. important::
 For the moment, the `Ktable` for the mix inherits attributes (such as the name of the molecule)
 from the most leftward `Ktable` in the operation. 

The renormalization operation also accepts an array of volume mixing ratio, but only if it has the dimensions `(Number of P points, number of T points)`.

In [ ]:
x_h2o=np.array([0.01+0.02*logP+0.00001*T for logP in h2o.logpgrid for T in h2o.tgrid]).reshape((h2o.Np,h2o.Nt))
mix=x_h2o*h2o+co2*0.02

# <span style="color:blue">Dealing with mixes: The `Kdatabase()` object </span>

## Loading a `Kdatabase()` object

### Global loading from files

A `Kdatabase()` object mainly contains a dictionary of `Ktable()` objects identified by the name of their molecule (in the attribute `self.ktables`, but they can be accessed with `self['mol_name']`). In addition, most of the attributes of the `Ktable()` are reloaded as attributes of `Kdatabase()`.

To load a database, just specify a list of molecules and a string filter that is common to all your filenames.

In [ ]:
molecs=['H2O','CO2']
filt='R300_0.3-50mu.ktable.SI'
database=xk.Kdatabase(molecs, filt)
database

It will look for files starting with the molecule names, followed by `_`, `.`, and `-`, and containing the filters.

If you want the moleculer name to be followed by something else than these characters, then just change the setting by using the `xk.Settings().set_delimiters` method.

.. important::
 The latest release of Exomol data is using a new convention to write molecule names (to include isotopologues) which is not yet completely supported by the Loading method above. We are working on that (so be sure to use the latest version). For the moment, to load these files into a database, you can use the method below.

Of course, this is unpractical when you do not have a homogeneous set of Ktables to start with. Then you can just feed a dictionary of molecules with the associated files path.

In [ ]:
database=xk.Kdatabase({'H2O':'data/corrk/H2O_R300_0.3-50mu.ktable.SI.hdf5',
                       'CO2':'data/corrk/CO2_R300_0.3-50mu.ktable.SI.pickle'})
database

### Adding individual, already-loaded `Ktable`s 

Sometimes, you want to modify/remap/bin_down your `Ktable`s before loading it into a `Kdatabase`. However, you do not necessarily want to have to store these intermediate steps into a file to read it into a database as described above.

To do this, you can add one or several `Ktable` using the `add_ktables(*ktables)` method.
If you want to start from scratch, you'll need to create an empty `Kdatabase` as follows.

In [ ]:
database=xk.Kdatabase(None) # create empty database
print(database)

h2o=xk.Ktable('H2O_R300_0.3-50mu.ktable.SI', remove_zeros=True)
co2=xk.Ktable('CO2_R300_0.3-50mu.ktable.SI', remove_zeros=True)
co2.remap_logPT(logp_array=np.arange(0,5),t_array=np.arange(200.,500.,50.))

database.add_ktables(h2o, co2)
print(database)

### Dealing with inhomogeneous databases

As we are loading `Ktables` from various possible sources, that we may have modified however we like, we are bound to end up with a inhomogeneous database because all the `Ktable`:

  * do not have the same datatype.
  * do not have the same weights and g grid,
  * do not have the same logP-T grid,
  * do not have the same spectral grid,
  * do not have the same units

For the moment, a `Kdatabase` needs to be homogeneous to be used to create mixes or compute forward models (see below).

The 1st issue is due to the fact that `exo_k` can handle other types of data (e.g. `Xtable` that are regular cross section tables; see dedicated section below). Although these data types can be handled in a database, a `Kdatabase` can only deal with a single type at a time. Loading different types will result in an error.  

The 2nd issue is delicate because there are no easy way to accurately re-interpolate a `Ktable` on a different g grid. So for the moment, there is no way to remedy this. The user needs to provide `Ktable` with the same g grid and weights. Loading tables with different g grids will result in an error. 
  
 The remaining issues are easily handled by following the suggestion in the warning message (see above):
 
   * Using `Kdatabase.remap_logPT(logp_array= , t_array= )` will homogenize the log P-T grid by
     interpolating (in place) all the `Ktables` to a common grid.
   * Using `Kdatabase.bin_down(wnedges= , **kwargs)` will homogenize the spectral grid by
     binning down (in place) all the `Ktables` to a common grid.
   * If you are using cross sections, the spectral grid can be homogenized using
     `Kdatabase.sample(wngrid= )` instead of bin_down (see below).
   * All the `Ktable`s can be converted to the same units using `Kdatabase.convert_p_unit()` and
     `Kdatabase.convert_kdata_unit()`, or directly `Kdatabase.convert_to_mks()`
  

In [ ]:
print(database)
print('remapping...')
database.remap_logPT(logp_array=np.arange(0,5),t_array=np.arange(200.,500.,50.))
print(database)

.. warning::
A `Ktable` is not deep copied when loaded into a database! Modifying the database inplace thus modifies the original table as well.

In [ ]:
# See how the values in the P and T grids have changed
# compared to their values in the previous section.
h2o

## Accessing individual `Ktables`

A `Kdatabase` is essentially a dictionary and individual `Ktable` objects can be accessed using the molecule name as a key. When adding `Ktable`s one by one, the molecule name used as a key is inherited from the `Ktable.mol` attribute. If you are loading a new `Ktable` with a molecule name that is already in the database, the latter will be replaced. 

In [ ]:
database['H2O'].filename

.. warning::
    You **cannot** load a new table this way!

```python
database['SiO']=SiO_ktable
TypeError: 'Kdatabase' object does not support item assignment
```

## Create a `Ktable()` for a mix of gases

One of the main interests of databases is to actually be able to create new correlated-k tables for mixtures of gases. 

Just create a dictionary with the names of the molecules and the corresponding volume mixing  ratios. Notice that the sum of the mixing ratios does not need to be equal to one. In that case, it is assumed that the rest of the gas is composed of radiatively inactive species. Then, the opacity given is in m^2 per total number of molecules (couting the inactive ones).

If a molecule is not in the database, it will just be considered radiatively inactive. 

In [ ]:
molecs=['H2O','CO2']
suffix='R300_0.3-50mu.ktable.SI'
database=xk.Kdatabase(molecs,suffix)

composition={'H2O':0.001,'CO2':400.e-6,'N2':'background'}

start=time.time()
mixH2O_CO2=database.create_mix_ktable(composition)

mixH2O_CO2.write_hdf5('data/corrk/mix_H2O_CO2')

end=time.time()
print("computation time ",end - start,'s')        

Here, the `'N2':'background'`in the composition dictionary tells the code that N2 is the background 'filler' gas. So whatever the vmr given before, N2 will be adjusted to get a total vmr of 1. Because N2 is not in the database, its opacity will however not be considered.

(Why doing this you might ask? Well, here including N2 does not change anything as it is considered transparent anyway. This will become important however when we will deal with atmospheric models where including N2 will allow us to have the right mean molecular weight for the atmosphere). 

In [ ]:
ptest=1.e4
ttest=300.
fig,axs=plt.subplots(1,2,sharex=True,sharey=True)
for species,x in composition.items():
  if x is not 'background':
    database[species].plot_spectrum(axs[0],p=ptest,t=ttest,g=1.,x=x,linestyle='dashed',yscale='log')
    database[species].plot_spectrum(axs[1],p=ptest,t=ttest,g=0.,x=x,linestyle='dashed',yscale='log')
mixH2O_CO2.plot_spectrum(axs[0],p=ptest,t=ttest,g=1.)
mixH2O_CO2.plot_spectrum(axs[1],p=ptest,t=ttest,g=0.)
axs[0].set_title('g=1')
axs[1].set_title('g=0')
for ax in axs :
    ax.set_xscale('log')
    ax.label_outer()
    ax.set_ylim(bottom=1.e-40)
fig.tight_layout()

## Binning down a `Kdatabase()`

Just like `Ktable()` objects, you can change the spectral resolution of an entire database with the same `bin_down(wnedges)` method. It will be applied to all ktables in place (`weights` keyword available as well). 

In [ ]:
R=10
wavenumber_grid=xk.wavenumber_grid_R(200.,10000.,R)
database.bin_down(wavenumber_grid)

Notice how the resolution has changed below when we run the exact same code. 

In [ ]:
ptest=1.e4
ttest=300.

fig,axs=plt.subplots(1,2,sharex=True,sharey=True)
for species,x in composition.items():
  if x is not 'background':
    database[species].plot_spectrum(axs[0],p=ptest,t=ttest,g=1.,x=x,linestyle='dashed',label=species)
    database[species].plot_spectrum(axs[1],p=ptest,t=ttest,g=0.,x=x,linestyle='dashed',label=species)
mixH2O_CO2.plot_spectrum(axs[0],p=ptest,t=ttest,g=1.,label='mix')
mixH2O_CO2.plot_spectrum(axs[1],p=ptest,t=ttest,g=0.,label='mix')
axs[0].set_title('g=1')
axs[1].set_title('g=0')
for ax in axs :
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.label_outer()
    ax.set_ylim(bottom=1.e-40)
    ax.legend()
fig.tight_layout()

# <span style="color:blue">Loading and creating correlated-k tables to use with LMDZ GCM</span>

## Creating new corr-k for LMDZ-GCM

`exo_k` has also been designed to create corr-k tables to be used with the LMDZ gcm.  

To produce a LMDZ usable directory with the corr-k and all the supporting files, nothing is simpler. 

.. important::
        This section only deals with corr-k tables *without* variable gases.
        For these, we need to deal with gas mixes and
        `Ktable5D` that are discussed a little further down.

First, let's create a grid for our IR and VI channels and create our two ktables.

In [ ]:
R=10
IR_grid=xk.wavenumber_grid_R(220.,10000.,R)
VI_grid=xk.wavenumber_grid_R(5200.,33000.,R)

IR_h2o_ktab=h2o_ktab_SI.bin_down_cp(IR_grid)

VI_h2o_ktab=h2o_ktab_SI.bin_down_cp(VI_grid)

Once your low resolution corrk table has been produced, just run

In [ ]:
IR_h2o_ktab.write_LMDZ('data/corrk/lmdz/new_h2o_corrk',band='IR')
VI_h2o_ktab.write_LMDZ('data/corrk/lmdz/new_h2o_corrk',band='VI')

**Should you care about units?**  

Actually... no! Because we have tracked units from the beginning, and we know LMDZ likes ktables in cm^2/molec and pressures in log mBar, **the conversion has been done automatically** for you.

The LMDZ aficionados may have noticed that in the `corrk/new_h2o_corrk` directory, two subdirectories have been created with the names `IR self.Nw` and `VI self.Nw` that contain the corr-k tables whereas we would like a single directory called `Number_of_IR_bins x Number_of_VI_bins`. Then just run:

In [ ]:
xk.finalize_LMDZ_dir('data/corrk/lmdz/new_h2o_corrk',IR_h2o_ktab.Nw,VI_h2o_ktab.Nw)

You just have to create a `Q.dat` file inthe directory and you are good to run LMDZ with new corrk.

## Loading existing LMDZ corr-k tables

To load a LMDZ corr-k table, the initialization method needs some more information:

  * The directory to use (`path=`)
  * The resolution (`res=`)
  * Whether you want to load the 'IR' or 'VI' band (`band=`)
  
The `mol` keyword can be given to specify which species is described. This will be useful for mixes. 

Here, if you do not specify units, you will keep the LMDZ ones (cm^2 and mBar). But we advise to use the SI units. In particular because our radiative transfer method use SI units. 

In [ ]:
IR_h2o_ktab=xk.Ktable(path='data/corrk/lmdz/new_h2o_corrk',res='39x19',band='IR',mol='H2O',
                kdata_unit='m^2',p_unit='Pa')
print(IR_h2o_ktab)

## Creating a LMDZ corr-k with a variable gas from a `Kdatabase`: the `Ktable5d`

Here, we will finally see one of the strengths of `Kdatabase` objects. Because we can create mix of gases and bin down, we have everything we need to create LMDZ like ktables for variable gases.

For this, use
```python
Kdatabase.create_mix_ktable5d(vgas_comp=, bg_comp=, x_array=)
```
where:

  - `vgas_comp` must contain a dictionary of the composition of the variable gas (it can be a single molecule).
  - `bg_comp` does the same for the background gas.
  - `x_array` is an array of the vol mix ratios of the variable gas that we want to compute in the table. 

In [ ]:
start=time.time()

molecs=['H2O','CO2']
suffix='R300_0.3-50mu.ktable.SI'

database=xk.Kdatabase(molecs,suffix)
# we first remap on a smaller logP T grid for speed
database.remap_logPT(logp_array=np.arange(6.), t_array=[200.,300.,400.,500.,600.])
x_array=[1.e-7,0.1]
vgas={'H2O':'background'}
bg_gas={'CO2':4.e-4,'N2':'background'}
mix_var_gas=database.create_mix_ktable5d(vgas_comp=vgas,bg_comp=bg_gas,x_array=x_array)
end=time.time()
print("computation time ",end - start)        

Then we can bin down before writing to file

In [ ]:
R=10
IR_grid=xk.wavenumber_grid_R(220.,10000.,R)
VI_grid=xk.wavenumber_grid_R(5200.,33000.,R)

IR_mix_var_gas_ktab=mix_var_gas.copy()
IR_mix_var_gas_ktab.bin_down(IR_grid)

VI_mix_var_gas_ktab=mix_var_gas.copy()
VI_mix_var_gas_ktab.bin_down(VI_grid)

IR_mix_var_gas_ktab.write_LMDZ('data/corrk/lmdz/new_earth_h2o-var',band='IR')
VI_mix_var_gas_ktab.write_LMDZ('data/corrk/lmdz/new_earth_h2o-var',band='VI')

xk.finalize_LMDZ_dir('data/corrk/lmdz/new_earth_h2o-var',IR_mix_var_gas_ktab.Nw,VI_mix_var_gas_ktab.Nw)

Let's plot the result compared to the initial cross sections at two water mixing ratios

In [ ]:
ptest=100.
ttest=500.
fig,axs=plt.subplots(1,2,sharex=True,sharey=True)
database['H2O'].plot_spectrum(axs[0],p=ptest,t=ttest,g=1.,x=x_array[0],label='H2O')
database['CO2'].plot_spectrum(axs[0],p=ptest,t=ttest,g=1.,x=bg_gas['CO2'],label='CO2')
IR_mix_var_gas_ktab.plot_spectrum(axs[0],p=ptest,t=ttest,x=x_array[0],g=1.,label='mix/IR')
VI_mix_var_gas_ktab.plot_spectrum(axs[0],p=ptest,t=ttest,x=x_array[0],g=1.,label='mix/VI')
database['H2O'].plot_spectrum(axs[1],p=ptest,t=ttest,g=1.,x=x_array[-1],label='H2O')
database['CO2'].plot_spectrum(axs[1],p=ptest,t=ttest,g=1.,x=bg_gas['CO2'],label='CO2')
IR_mix_var_gas_ktab.plot_spectrum(axs[1],p=ptest,t=ttest,x=x_array[-1],g=1.,label='mix/IR')
VI_mix_var_gas_ktab.plot_spectrum(axs[1],p=ptest,t=ttest,x=x_array[-1],g=1.,label='mix/VI')
axs[0].set_title('$x_{H2O}=10^{-7}$')
axs[1].set_title('$x_{H2O}=10^{-1}$')
for ax in axs :
    ax.set_ylim(bottom=1.e-40)
    ax.set_yscale('log')
    ax.set_xscale('log')
    ax.legend()
fig.tight_layout()

## Adding the opacity of a trace species to an existing corr-k table

If you want to add the opacity of a given molecule to a corr-k table (whether is has variable gas or not), it is rather simple. Just load your table and add the species you want:

In [ ]:
IR_mix_var_gas_ktab=xk.Ktable5d(path='data/corrk/lmdz/new_earth_h2o-var',res='39x19',band='IR',
                kdata_unit='m^2',p_unit='Pa')
co2=xk.Ktable('CO2_*R300_0.3-50mu.ktable.SI', remove_zeros=True)
co2.remap_logPT(t_array=IR_mix_var_gas_ktab.tgrid, logp_array=IR_mix_var_gas_ktab.logpgrid)
co2.bin_down(wnedges=IR_mix_var_gas_ktab.wnedges)
earth_plus_co2=IR_mix_var_gas_ktab+1.e-1*co2
print(earth_plus_co2)

In [ ]:
ptest=100.
ttest=500.
x_array=IR_mix_var_gas_ktab.xgrid
fig,ax=plt.subplots(1,1,sharex=True,sharey=True)
IR_mix_var_gas_ktab.plot_spectrum(ax,p=ptest,t=ttest,x=x_array[1],g=1.,label='mix')
co2.plot_spectrum(ax,p=ptest,t=ttest,x=0.1,g=1.,label='CO2')
earth_plus_co2.plot_spectrum(ax,p=ptest,t=ttest,x=x_array[1],g=1.,label='mix+0.01*CO2')
ax.set_ylim(bottom=1.e-40)
ax.set_yscale('log')
ax.set_xscale('log')
ax.legend()
fig.tight_layout()

# <span style="color:blue">Creating corr-k from `High_resolution` spectra</span>

.. important::
    Many cells in this part of the tutorial are not executable because the data needed to run them
    are not in the sample `data` directory provided.

## A note on ascii vs hdf5 data

The native ascii format for many high-resolution spectra (e.g. Kspectrum)
is highly inefficient and takes a lot of space.
Some Kspectrum files take more than a minute to read where the hdf5 equivalent would take a second. 
Keeping it will be the bottleneck of all 
the following computation. 

So we recommend you seriously consider converting your files to hdf5 before anything else.

If you have a bunch of file called `k001`, `k002`, `k003`, etc, `k0N` (let's take N=63 for exemple)
in `path_in` you might want to use something like

```python
path_in='/Users/jleconte/root/data2/jaziri/exo_k/spectrum_data/o3_step_band/'
path_out='/Users/jleconte/root/data1/leconte/atmosphere/radiative_transfer/hires_spectra/yassin_O3_hdf5/'
for i in np.arange(1,64):
    file='k'+str(i).zfill(3)
    xk.convert_kspectrum_to_hdf5(path_in+file,path_out+file+'.h5', wn_column=None,
        kdata_column=None, data_type='abs_coeff',
        file_kdata_unit='unspecified', kdata_unit='unspecified', skiprow=0)
```

The various available arguments are:

 * `file_kdata_unit`:
       Specifies the unit for the opacity data in the file. 
       The type of quantity may differ whether we are handling cross sections (surface)
       or absorption coefficients (inverse length)
  * `data_type`: 'xsec' or 'abs_coeff'.
       Whether the data read are cross-sections or absorption coefficients. Make sure you specify this if you changed
       the default `kdata_column`.
 * `wn_column`/`data_column`:
       Number of column to be read for wavenumber and kdata in python convention
       (0 is first, 1 is second, etc.)
 * `kdata_unit`:
       Unit to convert to.
 * `mult_factor`:
       A multiplicative factor that can be applied to kdata (for example to correct for
       any dilution effect, or specific conversion).
 * `skiprows`:
       Number of header lines to skip. For the latest Kspectrum format,
       the header is skipped automatically.

.. important::
    here again, with HDF5 files, we are tracking the units and data type (xsec or abs_coeff).
    So conversion to the right units can be made here, or at the k-coeff creation stage.

For example, if you want to convert your absorption coefficients to a weird unit, you can do
```python
path_in='/Users/jleconte/root/data2/jaziri/exo_k/spectrum_data/o3_step_band/'
path_out='/Users/jleconte/root/data1/leconte/atmosphere/radiative_transfer/hires_spectra/yassin_O3_hdf5/'
for i in np.arange(1,64):
    file='k'+str(i).zfill(3)
    xk.convert_kspectrum_to_hdf5(path_in+file,path_out+file+'.h5', data_type='abs_coeff',
        file_kdata_unit='m^-1', kdata_unit='cm^-1')
```

If you want to use the cross-section of a given species in a Ksectrum file (4nd column for example=>index=3) instead of the absorption coefficient and convert them to mks, you can do
```python
path_in='/Users/jleconte/root/data2/jaziri/exo_k/spectrum_data/o3_step_band/'
path_out='/Users/jleconte/root/data1/leconte/atmosphere/radiative_transfer/hires_spectra/yassin_O3_hdf5/'
for i in np.arange(1,64):
    file='k'+str(i).zfill(3)
    xk.convert_kspectrum_to_hdf5(path_in+file,path_out+file+'.h5', wn_column=None,
        kdata_column=3, data_type='xsec', skiprow=0,
        file_kdata_unit='cm^2/molecule', kdata_unit='m^2/molecule')
```
Note the last `kdata_unit` is superfluous if you have set `exo_k.Settings().set_mks(True)`

## Organizing your spectra into a grid

Now that you spent a few cpu x hours to compute some high resolution spectra, you would like to create a corr-k matrix with it.  

For this you'll need to provide the `logpgrid` and `tgrid` (and potentially `xgrid`) arrays onto which your spectra have been computed.

.. warning::
    By default, log pressures are specified in Pa in logpgrid!!! If you want
    to use another unit, do not forget to specify it with the grid_p_unit keyword.


You'll also need to provide the corresponding spectra filenames in an array of shape `(logpgrid.size, tgrid.size (, xgrid.size))` with the following order:
```python
[['spectrum for logP1,T1', 'spectrum for logP1,T2'],
       ['spectrum for logP2,T1', 'spectrum for logP2,T2'],
       ['spectrum for logP3,T1', 'spectrum for logP3,T2']]
```
or
```python
[[['spectrum for logP1,T1,x1', 'spectrum for logP1,T1,x2',
         'spectrum for logP1,T1,x3'],
        ['spectrum for logP1,T2,x1', 'spectrum for logP1,T2,x2',
         'spectrum for logP1,T2,x3']],

       [['spectrum for logP2,T1,x1', 'spectrum for logP2,T1,x2',
         'spectrum for logP2,T1,x3'],
        ['spectrum for logP2,T2,x1', 'spectrum for logP2,T2,x2',
         'spectrum for logP2,T2,x3']],

       [['spectrum for logP3,T1,x1', 'spectrum for logP3,T1,x2',
         'spectrum for logP3,T1,x3'],
        ['spectrum for logP3,T2,x1', 'spectrum for logP3,T2,x2',
         'spectrum for logP3,T2,x3']]]
```

### Kspectrum files generated for LMDZ

If you are using Kspectrum files generated for LMDZ, your files should be called k001, k002, etc. 

In the case of 4 pressure points, and 2 Temperature points, you can create the list of files in the right order as follows:

In [ ]:
xk.create_fname_grid_Kspectrum_LMDZ(4,2,suffix='')

If you have a variable gas with 3 different volume mixing ratios, then use

In [ ]:
xk.create_fname_grid_Kspectrum_LMDZ(4, 2, 3, suffix='')

And if you have converted them to hdf5, as recommended above, you can then use

In [ ]:
xk.create_fname_grid_Kspectrum_LMDZ(4, 2, 3, suffix='.h5')

### Files named with information on P, T (and x) in the title

You can create your grid of filenames by hand. But if the names are regular enough, the following function may help you.

See the 3 examples below:

In [ ]:
logpgrid=[1,10]
tgrid=[100,150,200]
file_grid=xk.create_fname_grid('spectrum_CO2_{p}Pa_{t}K.hdf5',
                  logpgrid=logpgrid, tgrid=tgrid, p_kw='p', t_kw='t')
file_grid

In [ ]:
logpgrid=[1,2]
tgrid=[100,150,200]
xgrid=[1.e-2,1.e-1]
file_grid=xk.create_fname_grid('O3-H2O_1e{logp}Pa_{t}K_vmr{x}.hdf5',
                  logpgrid=logpgrid, tgrid=tgrid, xgrid=xgrid, p_kw='logp', t_kw='t', x_kw='x')
file_grid

In [ ]:
logpgrid=[1,10,100,1000]
tgrid=np.array([100.,150.,200.])
file_grid=xk.create_fname_grid('Kspec_{logp}Pa_{t}K.hdf5',
                  logpgrid=logpgrid, tgrid=tgrid, p_kw='logp', t_kw='t')
file_grid

## Actually making the `Ktable`

Once we have our grids of pressure (in Pa), temperature (, vmr), and the grid of corresponding filenames, producing the `Ktable` is done like this

```python
temp=np.arange(100.,401.,50)
logpres=np.arange(-1,8)
file_grid=xk.create_fname_grid_Kspectrum_LMDZ(logpres.size, temp.size, suffix='.h5')
path='data/hires/yassin_O3_hdf5/'

#that's where you choose your wavenumber bin edges grid
wnedges=xk.wavenumber_grid_R(10.,10.1,980)
tmp=xk.hires_to_ktable(path=path, filename_grid=file_grid,
                    logpgrid=logpres, tgrid=temp, wnedges=wnedges, mol='O3')
tmp.remove_zeros() #if you do not want to have zeros in the table for interpolation purposes
print(tmp)
```
With these options, you create a table with 20 gauss point following the Gausse-Legendre quadrature. 

.. important::
    Whether data are in cross section or absorption coefficient form in the input data,
    they will be converted to cross sections to be put in the Ktable.
    As this conversion is done using the pressure grid you provide,
    it is important to specify its unit with the grid_p_unit keyword if you are NOT using Pa.

Hereafter are some variations around the same theme to show some commented examples of use. Pick the one that fits better to your use.

```python
temp=np.arange(100.,401.,50)
logpres=np.arange(-1,8)
path='data/hires/yassin_O3_hdf5/'

#that's where you choose your wavenumber bin edges grid
wnedges=xk.wavenumber_grid_R(10.,10.1,980)
tmp=xk.hires_to_ktable(path=path,
                    # because files organized as k001, k002, etc.
                    #  is the default, the above result can be
                    #  obtained without specifying a filename_grid.
                    #  You'll need to provide suffix here, however
                    suffix='.h5',
                    logpgrid=logpres, tgrid=temp, wnedges=wnedges, mol='O3')
tmp.remove_zeros() #if you do not want to have zeros in the table for interpolation purposes
print(tmp)
```

Here starting from ascii files and doing directly the conversion from abs_coeff to cross sections.
```python
temp=np.arange(100.,401.,50)
logpres=np.arange(-1,8)
path='data/hires/yassin_O3_hdf5/'
wnedges=xk.wavenumber_grid_R(10.,10.1,980)
tmp=xk.hires_to_ktable(path=path, data_type='abs_coeff',
                    logpgrid=logpres, tgrid=temp, wnedges=wnedges, mol='O3',
                    file_kdata_unit='m^-1', kdata_unit='m^2/molec')
tmp.remove_zeros() #if you do not want to have zeros in the table for interpolation purposes
print(tmp)
```

If you want to provide the g weights yourself, run this instead:
```python
temp=np.arange(100.,401.,50)
logpres=np.arange(-1,8)
path='data/hires/yassin_O3_hdf5/'
wnedges=xk.wavenumber_grid_R(10.,10.1,980)

weights=[my g weights]

tmp=xk.hires_to_ktable(path=path, suffix='.h5',
                    logpgrid=logpres, tgrid=temp, wnedges=wnedges, weights=weights, mol='O3')
tmp.remove_zeros() #if you do not want to have zeros in the table for interpolation purposes
```

You can now save your brand new Ktable on disk with any of the writing routines (`write_hdf5`, `write_LMDZ`, etc.).

## Corr-k with a variable gas: `Ktable5d`

If you have a grid of spectra with a gas that is varying along one dimension,
you just have to provide the grid of volume mixing ratios (`xgrid`) along with the other arguments. 

`hires_to_ktable` will automatically detect the new dimension and create a `Ktable5d` instead of a regular `Ktable` (see 4.3).

# <span style="color:blue">Dealing with cross sections: `Xtable()`objects </span>

Cross sections are like corr-k tables, except for the fact that they do not have a "g-dimension".

Thanks to the way `numpy` handles arrays and to the `python` inheritance system, handling a `Xtable()` is very similar to handling a `Ktable()` and you can do everything that you did before.  

It is so similar that you can even load a `Kdatabase()` with `Xtable()` objects. 

Just remember that the kdata array now has one less dimension. 

In [ ]:
xsecs=xk.Xtable('H2O.R10000.TauREx.pickle', kdata_unit='m^2', p_unit='Pa')
xsecs

In [ ]:
ptest=1.e2
ttest=300.
fig,ax=plt.subplots(1,1,sharex=True,sharey=False)  
xsecs.plot_spectrum(ax,p=ptest,t=ttest,yscale='log')
fig.tight_layout()

## `Xtable`: sample or bin_down

A unique feature of `Xtable`s over `Ktable`s is that there are two ways of changing the spectral grid:

    * `bin_down(wnedges= )`: does an area-conserving binning
      inside the bin edges provided as `wnedges`.
      The final spectral grid has a dimension `Nw=wnedges.size-1`.
    * `sample(wngrid= )`: just interpolates the cross sections
      on the given grid. The final spectral grid has a dimension `Nw=wngrid.size`.

## Loading `Xtable` with `kspectrum`-like spectra

You can also load your spectra to be used as cross sections.  

For this you'll need to provide the `logpgrid` and `tgrid` arrays onto which your spectra have been computed, as well as the corresponding spectra filenames in an array of shape `(logpgrid.size,tgrid.size)`

```python
path='data/hires/yassin_O3_hdf5/'
temp=np.arange(100.,401.,50)
logpgrid=np.arange(-1,8)
hires_o3_xsec=xk.hires_to_xtable(path=path,filename_grid=None, suffix='.h5',
                             logpgrid=logpgrid,tgrid=tgrid,
                             mol='O3',kdata_unit='m^2')
hires_o3_xsec.remove_zeros()
```

You can then save it to a hdf5 file with `write_hdf5()`.

## Creating a `Ktable` from an `Xtable`

As an `Xtable` represents monochromatic cross-sections, it is only natural that we should be able to produce `Ktable` with them. And we indeed can with the `Ktable.xtable_to_ktable()` method, or directly from the `__init__()` method with the `xtable=` keyword.

In [ ]:
R=20.
xsecs_h2o=xk.Xtable('H2O.R10000.TauREx.pickle', kdata_unit='m^2', p_unit='Pa')

ktab_h2o=xk.Ktable(xtable=xsecs_h2o, wnedges=xk.wavenumber_grid_R(1000.,20000.,R))

p_plot=1.e0
t_plot=300.
fig,ax=plt.subplots(1,1,sharex=False,sharey=False)
xsecs_h2o.plot_spectrum(ax, p=p_plot, t=t_plot, yscale='log', xscale='log', label='Original data')
ktab_h2o.plot_spectrum(ax, p=p_plot, t=t_plot, g=1., label='corr-k, g=1')
ktab_h2o.plot_spectrum(ax, p=p_plot, t=t_plot, g=0., label='corr-k, g=0')
ax.legend()
fig.tight_layout()

.. warning::
    Although in theory there is no formal criterion separating a cross section table
    and a set of high-resolution spectra, in practice, cross section table often
    have a moderate resolution, which makes them unsuitable for the computation
    of accurate k-coefficient tables. 

# <span style="color:blue">Dealing with continua: `Cia_table()` and `CIAdatabase()` objects </span>

Cia tables are like cross section tables, except for the fact that they do not have a pressure dimension.

They can still be loaded to be converted from one format to another (in particular HITRAN .cia files or water CKD files to HDF5), from one unit to another (our preferred unit is m^5/molec^2).

Finally, they will also be usefull to account for continua in opacity calculations and forward models below

But first, as for `Ktable`s, we need to setup the path where to look for cia tables.

In [5]:
xk.Settings().add_cia_search_path('data/cia')

Then you can load a `CIAdatabase` by providing the list of molecules you want to look for. It will try to load CIA data for as many pairs as it can find with these molecules.

In [ ]:
mols=['He','H2','N2','H2O']
cia_data=xk.CIAdatabase(molecules=mols)
cia_data

You can filter the files you want to look for by adding a string filter as the first argument(s)

In [ ]:
mols=['He','H2','N2','H2O']
cia_data=xk.CIAdatabase('2011', molecules=mols)
cia_data

If you only want a subset of these pairs, you can directly specify the pairs you want to look for with the `molecule_pairs` keyword. 

In [ ]:
mol_pairs=[['He','H2'],['N2','H2']]
cia_data=xk.CIAdatabase(molecule_pairs=mol_pairs)
cia_data

In [ ]:
xk.Settings().set_mks(True)
mols=['He','H2','N2']
cia_data=xk.CIAdatabase(molecules=mols)
ttest=300.
fig,axs=plt.subplots(2,1,sharex=False,sharey=False)  
cia_data['H2']['H2'].plot_spectrum(axs[0],t=ttest,x_axis='wns',yscale='log',label='H2-H2')
cia_data['H2']['He'].plot_spectrum(axs[0],t=ttest,x_axis='wns',label='H2-He')
cia_data['N2']['H2'].plot_spectrum(axs[1],t=ttest,x_axis='wns',yscale='log',label='N2-H2')
cia_data['N2']['N2'].plot_spectrum(axs[1],t=ttest,x_axis='wns',yscale='log',label='N2-N2')
for ax in axs:
    ax.legend()
fig.tight_layout()

Because CIA are often on very inhomogeneous grids (as above), you will need to sample them on the grid of your `Ktable` or `Xtable` to use them.

In [ ]:
ref_wns_grid=h2o_ktab_SI.wns
cia_data.sample(ref_wns_grid)
fig,axs=plt.subplots(2,1,sharex=False,sharey=False)  
cia_data['H2']['H2'].plot_spectrum(axs[0],t=ttest,x_axis='wns',yscale='log',label='H2-H2')
cia_data['H2']['He'].plot_spectrum(axs[0],t=ttest,x_axis='wns',label='H2-He')
cia_data['N2']['H2'].plot_spectrum(axs[1],t=ttest,x_axis='wns',yscale='log',label='N2-H2')
cia_data['N2']['N2'].plot_spectrum(axs[1],t=ttest,x_axis='wns',yscale='log',label='N2-N2')
for ax in axs:
    ax.legend()
fig.tight_layout()

The method:
```python
CIAdatabase.cia_cross_section(pressure,temperature,composition)
```
can be used to compute the effective CIA cross-section for the whole gas (in m^2 per average molecule)


In [ ]:
compo={'H2':0.2,'He':0.1,'N2':0.4}
raw=cia_data.cia_cross_section([1.,3.],[300.,500.],compo)
fig,axs=plt.subplots(2,1,sharex=False,sharey=False,figsize=(6,5))  
cia_data['H2']['H2'].plot_spectrum(axs[0],t=ttest,x_axis='wns',yscale='log')
cia_data['H2']['He'].plot_spectrum(axs[0],t=ttest,x_axis='wns')
cia_data['N2']['H2'].plot_spectrum(axs[0],t=ttest,x_axis='wns',yscale='log')
axs[1].plot(cia_data.wns,raw[0])
axs[1].set_yscale('log')
axs[1].set_ylabel('Cross section (m^2/molec)')
axs[1].set_xlabel('Wavenumber')
fig.tight_layout()

# <span style="color:blue">How to use `exo_k` as an opacity interpolator inside your code: The `Gas_mix` object</span>

`Exo_k` can also be used to directly access and interpolate opacity within your code. For this, one possibility is to load and access individual K/X/CIA tables and add all the opacities. 

Another is to use the `Gas_mix` object: it contains the information on the pressure/temperature conditions of the gas, its composition, and can be connected to radiative databases to compute the opacity of the gas from all contributions. 

## Composition and background gas

An empty `Gas_mix` object can be instanciated as follows and filled later on.

In [ ]:
gas=xk.Gas_mix()
print(gas)

The message shows that, at this time, the `Gas_mix` is empty. Effectively, it is filled with 'inactive_gas'.

Composition is specified using a dictionary where molecules are the keys and the volume mixing ratios as values. The rest of the `Gas_mix` is filled with inactive gas so that Sum(vmr)=1.

In [ ]:
gas.set_composition({'CO2':0.01})
print(gas)
gas.set_composition({'N2':0.8})  # reset the whole composition
print(gas)

One of the gases can have a vmr equal to 'background'.
In this case, the vmr of this gas will always be updated to be 1.-Sum(other vmrs).

In [ ]:
gas=xk.Gas_mix({'CO2':0.01,'N2':'background'}) # can also directly instantiate with a composition
print(gas)
gas['O2']=0.2  # only adds a gas or changes its vmr.
print(gas)
gas['H2O']=[0.01, 0.001]  #  Accepts arrays
print(gas)

## Getting the molar mass

The molar mass (in kg/mol) of your mix can be computed simply using:

In [ ]:
gas.set_composition({'N2':'background','CO2':340.e-6,'H2O':0.001})
print(gas.molar_mass())
gas.set_composition({'CH3COOH':'background'})
print(gas.molar_mass())
gas.set_composition({'Mg3SiO4':'background'})
print(gas.molar_mass())
gas=xk.Gas_mix({'N2':'background','CO2':340.e-6,'H2O':[0.001,0.1]})
print(gas.molar_mass())

## Connecting a gas with a radiative database

To connect with radiative data, first you need to load the databases you want to use. You can combine a `CIAdatabase` with a `Kdatabase` filled with either `Ktable`s or `Xtable`s (but not both). 

Then these database can be added to a `Gas_mix`, or you can directly instantiate with these databases. This only needs to be done once.

.. warning::
    With `Gas_mix`, radiative databases need to be converted in SI units to be used. The simplest way to achieve that is to use `exo_k.Settings().set_mks(True)`.

In [ ]:
database=xk.Kdatabase(['CO2','H2O'],'R300_0.3-50mu.ktable.SI')

gas.set_k_database(database)

#Loads CIA and sample them on the right spectral grid.
cia_data=xk.CIAdatabase(molecules=['N2','H2O'], mks=True)
cia_data.sample(database.wns)

gas.set_cia_database(cia_data)
print(gas)

Or everything can be done at initialization

In [ ]:
database=xk.Kdatabase(['CO2','H2O'],'R300_0.3-50mu.ktable.SI')

#Loads CIA and sample them on the right spectral grid.
cia_data=xk.CIAdatabase(molecules=['N2','H2O'], mks=True)
cia_data.sample(database.wns)

gas=xk.Gas_mix(composition={'N2':'background','CO2':340.e-6,'H2O':[0.001,0.1]},
               k_database=database, cia_database=cia_data)
print(gas)

Now, the effective cross section on the gas can be computed as many times as necessary by calling the `Gas_mix.cross_section()` method providing log pressure (in Pa), temperature, and composition.

In [ ]:
logp_array=6.
t_array=300.
gas.set_composition({'CO2':0.01,'N2':'background'})
# composition can be set once and for all and only logP and T or changed at call time.
kdata=gas.cross_section(logp_array=logp_array, t_array=t_array)
fig,ax=plt.subplots(1,1,sharey=False,figsize=(8,4))  
ax.plot(10000./gas.wns,kdata[0,:,-1])
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_ylabel('Cross section (m^2/molec)')
ax.set_xlabel('Wavelength (micron)')
fig.tight_layout()

In [ ]:
logp_array=2.
t_array=300.
composition={'N2':'background','CO2':340.e-6,'H2O':1.e-3}
kdata=gas.cross_section(composition=composition, # composition can also be specified at call time
                    logp_array=logp_array, t_array=t_array)

fig,ax=plt.subplots(1,1,sharey=False,figsize=(8,4))  
ax.plot(10000./gas.wns,kdata[0,:,-1])
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_ylabel('Cross section (m^2/molec)')
ax.set_xlabel('Wavelength (micron)')
fig.tight_layout()

If you need to call this function for many logP-T points, it is more efficient to call the function once, providing logP-T (and even vmr) arrays

In [ ]:
logp_array=np.linspace(1.,6.,100)
t_array=300.*np.ones(logp_array.size)
composition={'N2':'background','CO2':340.e-6,
             'H2O':0.001*np.ones(logp_array.size)}
kdata=gas.cross_section(composition=composition,
                    logp_array=logp_array, t_array=t_array)
print(kdata.shape)

## Choosing gas recombination method (when using `Ktable`s)

Two recombination methods are available when several gases are present:

  * Simple addition: Fast. It is ok at moderate resolution
    (and if you are not looking for ppm level accuracy), but can be inacurrate
    if you are using large bins that encompass molecular bands of several molecules.
    This is the default mode for `Gas_mix.cross_section()`.
  * Random overlap: Very accurate in many situations, but a bit longer,
    especially if you have many molecules. This is the default mode for regular `Ktable`
    recombinations.

For `Gas_mix.cross_section()`, the behavior can be controlled with the `random_overlap=True/False` keyword.

In [ ]:
logp_array=2.
t_array=300.
gas.set_composition({'N2':'background','CO2':340.e-6,'H2O':0.001})
gas.set_logPT(logp_array=logp_array, t_array=t_array)

kdata=gas.cross_section()
kdata_randoverlap=gas.cross_section(random_overlap=True)

%timeit -r 10 -n 10 gas.cross_section()
%timeit -r 10 -n 10 gas.cross_section(random_overlap=True)


fig,ax=plt.subplots(1,1,sharey=False,figsize=(8,4))  
ax.plot(10000./gas.wns,kdata_randoverlap[0,:,-1], label='Rand. Overlap')
ax.plot(10000./gas.wns,kdata[0,:,-1], label='addition')
ax.set_yscale('log')
ax.set_xscale('log')
ax.legend()
ax.set_xlabel('Wavelength (micron)')
ax.set_ylabel('Cross section (m^2/molec)')
fig.tight_layout()

You can try any molecule with a regular Element Number of atoms notation. 

# <span style="color:blue">The `Atm`class: An integrated atmospheric radiative-transfer model</span>

Whether you want to test various assumptions (will my resolution be enough, are my initial data adequate, what weights should I use, etc.) or directly simulate the transmission/emission spectrum of a planet, we implemented a simple 1d atmosphere model so that you can do just that.

This revolves around the `Atm()` class, which defines an atmosphere with Pressure, Temperature, and composition profiles. In essence, we are using the structure of the `Gas_mix` object and add the additional attributes that make an atmosphere more than just some gas with a given pressure, temperature, and composition: a gravity, a planetary radius, etc. 

As `Xtable()` objects can be used in a `Kdatabase()`, you can do the equivalent of Line by Line radiative transfer. 

If you want to start from very high-res spectra, first load them into a `Xtable()`, and then into a Database.

## Emission

In [ ]:
database=xk.Kdatabase(['CO2'],'R300_0.3-50mu.ktable.SI')

T0=200.
atm_ck=xk.Atm(psurf=640.e0, ptop=1.e-1, Tsurf=T0, Tstrat=100., grav=3.4,
                    composition={'CO2':'background'}, Nlev=30, k_database=database)

FluxTop=atm_ck.emission_spectrum(integral=True, wl_range=[0.1,150.0])
SurfFlux=atm_ck.surf_bb(integral=True)
TopBB=atm_ck.top_bb(integral=True)
print("Forward model duration:")
%timeit -n 2 -r 10 atm_ck.emission_spectrum(integral=True, wl_range=[0.1,150.0])

The `integral=True` keyword specifies that the source function is integrated inside each bin. If `False`, the planck function is just evaluated at the center of the bin. It is faster, and ok for thin bins, but inacurate for large ones.

The `wl_range` range keyword allows you to limit the wavelength range onto wich you want the calculation to be done to save time. 

In [ ]:
print(FluxTop.total,'W/m^2 (BB at tsurf over the computed nu range is ',SurfFlux.total,'W/m^2)')
print('(sigma*T**4 is ',xk.SIG_SB*T0**4,'W/m^2)')

In [ ]:
fig,axs=plt.subplots(1,2,sharey=False,figsize=(8,4))  
SurfFlux.plot_spectrum(axs[0],label='Blackbody ($T_{surf}$)')
TopBB.plot_spectrum(axs[0],label='Blackbody ($T_{strat}$)')
FluxTop.plot_spectrum(axs[0],label='OLR')
SurfFlux.plot_spectrum(axs[1],per_wavenumber=False,label='Blackbody ($T_{surf}$)')
TopBB.plot_spectrum(axs[1],per_wavenumber=False,label='Blackbody ($T_{strat}$)')
FluxTop.plot_spectrum(axs[1],per_wavenumber=False,label='OLR')

axs[0].set_ylabel('Flux ($W/m^2/cm^{-1}$)')
axs[1].set_ylabel('Flux ($W/m^2/\mu m$)')
for ax in axs:
    ax.legend()
fig.tight_layout()

You can also bin it down, and when spectra have the same wavenumber grid, perform algeabric operations)

In [ ]:
R=100
wavenumber_grid=xk.wavenumber_grid_R(200.,10000.,R)
binned_spec=FluxTop.bin_down_cp(wavenumber_grid)

fig,axs=plt.subplots(1,2,sharey=True)  
FluxTop.plot_spectrum(axs[0],label='OLR')
binned_spec.plot_spectrum(axs[0],label='binned OLR')
(SurfFlux-FluxTop).plot_spectrum(axs[1],label='Surf Blackbody - OLR')

for ax in axs:
    ax.set_ylabel('Flux ($W/m^2/cm^{-1}$)')
    ax.legend()
fig.tight_layout()

In [ ]:
origin = 'upper'
delta = 0.025

x =database.wls
y =atm_ck.play[:]
X, Y = np.meshgrid(x, y)
Z = atm_ck.exp_minus_tau()

nr, nc = Z.shape

fig, ax = plt.subplots(constrained_layout=True)
ax.invert_yaxis()
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim(left=0.3,right=20.)
CS = ax.contourf(X, Y, Z, 10, cmap=plt.cm.bone, origin=origin)
ax.set_title('Atm. Transmittance')
ax.set_xlabel('Wavelength (mu)')
ax.set_ylabel('Log(Pressure/Pa)')
# Make a colorbar for the ContourSet returned by the contourf call.
cbar = fig.colorbar(CS)
cbar.ax.set_ylabel('Exp(-tau)')
# Add the contour line levels to the colorbar

## Emission with continuum and your own thermal profile

The `Atm` object can also be initialized with a custom `logplev` and `tlev` profile. The vmr of the molecules can also be specified as a profile. 

.. warning::
    The (log) pressure (in Pa) and temperature must be specified at the interfaces between the layers (the levels, with `Nlev=Nlay+1`).
    The surface is the bottom level. But opacities are computed at the layers midpoints. The pressure and temperature of a layer are computed using respectively the geometric and arithmetic means of the values at levels. The vmr must be specified at the layers (so `logplev.size=vmr.size+1`).

In [ ]:
database=xk.Kdatabase(['CO2','H2O'],'R300_0.3-50mu.ktable.SI')

T0=320.;Tstrat=180.
#create the level presure grid 
logplev=np.linspace(-1,5,100)
#create the layer presure grid which is halfway in log
logplay=(logplev[1:]+logplev[:-1])/2.
# design your adiabat
tlev=T0*(10**logplev/1.e5)**0.28
tlev=np.where(tlev<Tstrat,Tstrat,tlev)
tlay=(tlev[1:]+tlev[:-1])/2.
# create your composition. Here, only water is variable and follows saturation.
# the composition must be given in the layers
xH2Olay=np.exp(-2257.e3*0.018/8.31* (1./tlay - 1./373.))*1.e5/10**logplay

#Loads CIA and sample them on the right spectral grid.
cia_data=xk.CIAdatabase(molecules=['N2','H2O'], mks=True)
cia_data.sample(database.wns)

#Loads the atmosphere and computes some intermediate things
# without CIA
atm_ck=xk.Atm(logplev=logplev,tlev=tlev,grav=9.81,
                    composition={'CO2':4.e-4,'H2O':xH2Olay,'N2':'background'},
                    k_database=database)
# with CIA
atm_ck_cont=xk.Atm(logplev=logplev,tlev=tlev,grav=9.81,
                    composition={'CO2':4.e-4,'H2O':xH2Olay,'N2':'background'},
                    k_database=database, cia_database=cia_data)

# Computes emission flux for the two cases
FluxTop=atm_ck.emission_spectrum(integral=True,wl_range=[0.1,40.0], mu_quad_order=3)
FluxTop_cont=atm_ck_cont.emission_spectrum(integral=True,wl_range=[0.1,40.0], mu_quad_order=3)

# Computes surface BlackBody
SurfFlux=atm_ck.surf_bb(integral=True)

# Computes top of atmosphere BlackBody
TopBB=atm_ck.top_bb(integral=True)
print(FluxTop.total,'W/m^2 (BB at tsurf over the computed nu range is ',SurfFlux.total,'W/m^2)')
print('(sigma*T**4 is ',xk.SIG_SB*T0**4,'W/m^2)')

In [ ]:
fig,axs=plt.subplots(1,2,sharey=False, figsize=(8,4))  
SurfFlux.plot_spectrum(axs[0],label=r'Blackbody ($T_{surf}$)')
TopBB.plot_spectrum(axs[0],label=r'Blackbody ($T_{strat}$)')
FluxTop.plot_spectrum(axs[0],label=r'OLR w/o CIA')
FluxTop_cont.plot_spectrum(axs[0],label=r'OLR w/ CIA')
SurfFlux.plot_spectrum(axs[1],per_wavenumber=False,label=r'Blackbody ($T_{surf}$)')
TopBB.plot_spectrum(axs[1],per_wavenumber=False,label=r'Blackbody ($T_{strat}$)')
FluxTop.plot_spectrum(axs[1],per_wavenumber=False,label=r'OLR w/o CIA')
FluxTop_cont.plot_spectrum(axs[1],per_wavenumber=False,label=r'OLR w/ CIA')

axs[0].set_ylabel(r'Flux ($W/m^2/cm^{-1}$)')
axs[1].set_ylabel(r'Flux ($W/m^2/\mu m$)')
for ax in axs:
    ax.legend()
#axs[0].invert_yaxis()
fig.tight_layout()

## Transmission

In [ ]:
T0=2000.;xH2O=1.e-5;Rs=1.*u.Rsun;grav=10.
database=xk.Kdatabase(['H2O'],'50mu','SI',remove_zeros=True)
cia_data=xk.CIAdatabase(molecules=['H2','He'], mks=True)
cia_data.sample(database.wns)

atm_ck=xk.Atm(psurf=10.e5,ptop=1.e-4,Tsurf=T0,Tstrat=T0,grav=grav,
                    composition={'H2':'background','H2O':xH2O},Nlev=100,Rp=1.*u.Rjup,
                    k_database=database, cia_database=cia_data)
spec=atm_ck.transmission_spectrum(Rstar=Rs, rayleigh=True)
print("Forward model duration:")
%timeit -n 2 -r 5 atm_ck.transmission_spectrum(Rstar=Rs, rayleigh=True)

In [ ]:
fig,ax=plt.subplots(1,1,sharex=True,sharey=True)  
spec.plot_spectrum(ax,xscale='log',label='Native corr-k resolution')
spec.bin_down_cp(xk.wavenumber_grid_R(400.,30000.,200)).plot_spectrum(ax,xscale='log',label=r'$R=200$')
spec.bin_down_cp(xk.wavenumber_grid_R(400.,30000.,30)).plot_spectrum(ax,xscale='log',label=r'$R=30$')
ax.set_ylabel('Depth')
ax.legend()
fig.tight_layout()

## Heating rates

In [ ]:
T0=300.
Tstrat=300.
database=xk.Kdatabase(['CO2','H2O'],'_R300_0.3-50mu.ktable.SI')
cia_data=xk.CIAdatabase(molecules=['H2','He'], mks=True)
cia_data.sample(database.wns,remove_zeros=True)

atm_ck=xk.Atm(psurf=1.e10,ptop=1.e-1,Tsurf=T0,Tstrat=Tstrat,grav=10.,
                    composition={'H2':'background','CO2':0.},
                    rcp=0.2183,Rp=1.*u.Rearth,
                    Nlev=30,
                    k_database=database,cia_database=cia_data)
HR=atm_ck.heating_rate(szangle=60.)
atm_ck.set_gas({'H2':'background','CO2':.05,'H2O':0.5})
HR2=atm_ck.heating_rate(szangle=60.)
HR3=atm_ck.heating_rate(szangle=89.99)

In [ ]:
fig,axs=plt.subplots(1,2,sharey=True)  
axs[0].set_yscale('log')
axs[0].set_xscale('log')
axs[0].plot(HR,atm_ck.play)
axs[0].plot(HR2,atm_ck.play)
axs[0].plot(HR3,atm_ck.play)
axs[1].plot(atm_ck.tlev,atm_ck.plev)
axs[0].set_xlabel('Heating Rate (K/s)')
axs[1].set_xlabel('Temperature (K)')
axs[0].set_ylabel('Pressure (Pa)')
axs[0].invert_yaxis()
axs[0].set_xlim(left=1.e-20,right=1.)
fig.tight_layout()